In [1]:
import torch
from helpers import model_summary
x_dummy = 2 * torch.rand(5, 2) - 1

In [2]:
print(x_dummy)

tensor([[-0.0859, -0.5638],
        [-0.9495, -0.5535],
        [-0.4666,  0.9139],
        [ 0.3270,  0.1868],
        [ 0.0832, -0.2958]])


In [3]:
from utils.positional_encodings import SphericalFourierFeatureTransform

In [8]:
model = SphericalFourierFeatureTransform(omegas=[30.0, 30.0], scale=[2, 2])

In [9]:
t_dummy = torch.tensor([[45.0, 60.0], [-90.0,-180.0]])
model(t_dummy)

tensor([[-0.9754, -0.3795,  0.2206, -0.9252,  0.2922,  0.5040,  0.9563, -0.8637,
         -0.2850, -0.4916, -0.9328,  0.8424, -0.1109, -0.1913, -0.3629,  0.3278,
          0.0645,  0.1112,  0.2110, -0.1905, -0.2704, -0.4663, -0.8848,  0.7991],
        [ 0.4303, -0.7022, -0.9027,  0.7120, -0.7769, -0.9999,  0.6297,  0.0139,
         -0.3343, -0.4303,  0.2710,  0.0060,  0.5455,  0.7021, -0.4421, -0.0097,
          0.7013,  0.9026, -0.5684, -0.0125, -0.5531, -0.7119,  0.4483,  0.0099]])

## SIREN and FINER MODEL

In [3]:
from models.implicit_neural_representations.inr_models.siren_model import SirenModel, FinerModel

In [4]:
# initialize siren model without residual layers
siren_model = SirenModel(
    in_features=2,
    out_features=2,
    hidden_layers=5,
    hidden_features=512,
    first_omega_0=30.0,
    hidden_omega_0=30.0,
    residual_net=False
)

In [5]:
output = siren_model(x_dummy)

In [6]:
# Print input and output
print("Input:")
print(x_dummy)
print("Output:")
print(output)

Input:
tensor([[ 0.7798,  0.4665],
        [-0.5352,  0.9371],
        [ 0.6600, -0.1392],
        [ 0.7714, -0.2785],
        [-0.4264,  0.6957]])
Output:
tensor([[-0.0682, -0.0505],
        [-0.0359,  0.0042],
        [ 0.0503,  0.0490],
        [ 0.0455, -0.0738],
        [-0.0148, -0.0271]], grad_fn=<MmBackward0>)


In [7]:
model_summary(siren_model, input_size=(1, 2))

Layer (type:depth-idx)                   Output Shape              Param #
SirenModel                               [1, 2]                    --
├─Sequential: 1-1                        [1, 2]                    --
│    └─SirenLayer: 2-1                   [1, 512]                  --
│    │    └─Linear: 3-1                  [1, 512]                  1,536
│    │    └─siren_activation: 3-2        [1, 512]                  --
│    └─SirenLayer: 2-2                   [1, 512]                  --
│    │    └─Linear: 3-3                  [1, 512]                  262,656
│    │    └─siren_activation: 3-4        [1, 512]                  --
│    └─SirenLayer: 2-3                   [1, 512]                  --
│    │    └─Linear: 3-5                  [1, 512]                  262,656
│    │    └─siren_activation: 3-6        [1, 512]                  --
│    └─SirenLayer: 2-4                   [1, 512]                  --
│    │    └─Linear: 3-7                  [1, 512]                  262,6

In [8]:
finer_model = FinerModel(
    in_features=2,
    out_features=2,
    bias=True,
    hidden_layers=5,
    hidden_features=128,
    first_omega_0=30.,
    hidden_omega_0=30.,
    first_k=10,
    hidden_k=10,
    residual_net=False
)

In [9]:
output = finer_model(x_dummy)

In [10]:
# Print input and output
print("Input:")
print(x_dummy)
print("Output:")
print(output)

Input:
tensor([[ 0.7798,  0.4665],
        [-0.5352,  0.9371],
        [ 0.6600, -0.1392],
        [ 0.7714, -0.2785],
        [-0.4264,  0.6957]])
Output:
tensor([[-0.0441, -0.0294],
        [ 0.0495,  0.0042],
        [ 0.0408,  0.0147],
        [ 0.0278,  0.0891],
        [-0.0059, -0.0038]], grad_fn=<MmBackward0>)


In [11]:
model_summary(finer_model, input_size=(1, 2))

Layer (type:depth-idx)                   Output Shape              Param #
FinerModel                               [1, 2]                    --
├─Sequential: 1-1                        [1, 2]                    --
│    └─FinerLayer: 2-1                   [1, 128]                  --
│    │    └─Linear: 3-1                  [1, 128]                  384
│    │    └─siren_activation: 3-2        [1, 128]                  --
│    └─FinerLayer: 2-2                   [1, 128]                  --
│    │    └─Linear: 3-3                  [1, 128]                  16,512
│    │    └─siren_activation: 3-4        [1, 128]                  --
│    └─FinerLayer: 2-3                   [1, 128]                  --
│    │    └─Linear: 3-5                  [1, 128]                  16,512
│    │    └─siren_activation: 3-6        [1, 128]                  --
│    └─FinerLayer: 2-4                   [1, 128]                  --
│    │    └─Linear: 3-7                  [1, 128]                  16,512
│ 

In [12]:
siren_residual_model = SirenModel(
    in_features=2,
    out_features=2,
    hidden_layers=5,
    hidden_features=512,
    first_omega_0=30.0,
    hidden_omega_0=30.0,
    residual_net=True
)

In [13]:
output = siren_residual_model(x_dummy)

In [14]:
# Print input and output
print("Input:")
print(x_dummy)
print("Output:")
print(output)

Input:
tensor([[ 0.7798,  0.4665],
        [-0.5352,  0.9371],
        [ 0.6600, -0.1392],
        [ 0.7714, -0.2785],
        [-0.4264,  0.6957]])
Output:
tensor([[-0.0044, -0.0489],
        [ 0.0034,  0.0613],
        [-0.0276, -0.0291],
        [-0.0632,  0.0422],
        [-0.0406, -0.0189]], grad_fn=<MmBackward0>)


In [15]:
model_summary(siren_residual_model, input_size=(1, 2))

Layer (type:depth-idx)                   Output Shape              Param #
SirenModel                               [1, 2]                    --
├─Sequential: 1-1                        [1, 2]                    --
│    └─SirenLayer: 2-1                   [1, 512]                  --
│    │    └─Linear: 3-1                  [1, 512]                  1,536
│    │    └─siren_activation: 3-2        [1, 512]                  --
│    └─SirenResidualLayer: 2-2           [1, 512]                  --
│    │    └─Linear: 3-3                  [1, 512]                  262,656
│    │    └─siren_activation: 3-4        [1, 512]                  --
│    │    └─Linear: 3-5                  [1, 512]                  262,656
│    │    └─siren_activation: 3-6        [1, 512]                  --
│    └─SirenResidualLayer: 2-3           [1, 512]                  --
│    │    └─Linear: 3-7                  [1, 512]                  262,656
│    │    └─siren_activation: 3-8        [1, 512]                  

In [16]:
finer_residual_model = FinerModel( 
    in_features=2,
    out_features=2,
    bias=True,
    hidden_layers=5,
    hidden_features=128,
    first_omega_0=30.,
    hidden_omega_0=30.,
    first_k=10,
    hidden_k=10,
    residual_net=True
)

In [17]:
output = finer_residual_model(x_dummy)

In [18]:
# Print input and output
print("Input:")
print(x_dummy)
print("Output:")
print(output)

Input:
tensor([[ 0.7798,  0.4665],
        [-0.5352,  0.9371],
        [ 0.6600, -0.1392],
        [ 0.7714, -0.2785],
        [-0.4264,  0.6957]])
Output:
tensor([[ 0.0354,  0.0146],
        [ 0.0188,  0.0157],
        [-0.0600,  0.0134],
        [-0.0120,  0.0125],
        [ 0.0328,  0.0117]], grad_fn=<MmBackward0>)


In [19]:
model_summary(finer_residual_model, input_size=(1, 2))

Layer (type:depth-idx)                   Output Shape              Param #
FinerModel                               [1, 2]                    --
├─Sequential: 1-1                        [1, 2]                    --
│    └─FinerLayer: 2-1                   [1, 128]                  --
│    │    └─Linear: 3-1                  [1, 128]                  384
│    │    └─siren_activation: 3-2        [1, 128]                  --
│    └─FinerResidualLayer: 2-2           [1, 128]                  --
│    │    └─Linear: 3-3                  [1, 128]                  16,512
│    │    └─siren_activation: 3-4        [1, 128]                  --
│    │    └─Linear: 3-5                  [1, 128]                  16,512
│    │    └─siren_activation: 3-6        [1, 128]                  --
│    └─FinerResidualLayer: 2-3           [1, 128]                  --
│    │    └─Linear: 3-7                  [1, 128]                  16,512
│    │    └─siren_activation: 3-8        [1, 128]                  --
│ 

## Gaussian Model and Finer Gaussian

In [20]:
from models.implicit_neural_representations.inr_models.gaussian_model import GaussianModel, GaussianFinerModel

In [21]:
# initialize gaussian model without residual layers
gaussian_model = GaussianModel(
    in_features=2,
    out_features=2,
    hidden_layers=5,
    hidden_features=512,
    scale=30.0,
    weight_init=2.0,
    bias_init=2.0,
    residual_net=False
)

In [22]:
output = gaussian_model(x_dummy)

In [23]:
# Print input and output
print("Input:")
print(x_dummy)
print("Output:")
print(output)

Input:
tensor([[ 0.7798,  0.4665],
        [-0.5352,  0.9371],
        [ 0.6600, -0.1392],
        [ 0.7714, -0.2785],
        [-0.4264,  0.6957]])
Output:
tensor([[ 1.2085,  0.1593],
        [ 1.7541,  2.7034],
        [-0.5871,  3.5373],
        [ 1.4023, -0.7308],
        [-2.2801, -2.7642]], grad_fn=<MmBackward0>)


In [24]:
model_summary(gaussian_model, input_size=(1, 2))

Layer (type:depth-idx)                             Output Shape              Param #
GaussianModel                                      [1, 2]                    --
├─Sequential: 1-1                                  [1, 2]                    --
│    └─GaussianFourierFeatureTransform: 2-1        [1, 4]                    --
│    └─GaussianLayer: 2-2                          [1, 512]                  --
│    │    └─Linear: 3-1                            [1, 512]                  2,560
│    │    └─gaussian_activation: 3-2               [1, 512]                  --
│    └─GaussianLayer: 2-3                          [1, 512]                  --
│    │    └─Linear: 3-3                            [1, 512]                  262,656
│    │    └─gaussian_activation: 3-4               [1, 512]                  --
│    └─GaussianLayer: 2-4                          [1, 512]                  --
│    │    └─Linear: 3-5                            [1, 512]                  262,656
│    │    └─gaussian_a

In [25]:
gaussian_residual_model = GaussianModel(
    in_features=2,
    out_features=2,
    hidden_layers=5,
    hidden_features=512,
    scale=30.0,
    weight_init=0.1,
    residual_net=True
)

In [26]:
output = gaussian_residual_model(x_dummy)

In [27]:
print("Input:")
print(x_dummy)
print("Output:")
print(output)

Input:
tensor([[ 0.7798,  0.4665],
        [-0.5352,  0.9371],
        [ 0.6600, -0.1392],
        [ 0.7714, -0.2785],
        [-0.4264,  0.6957]])
Output:
tensor([[ 0.1038,  0.1144],
        [-0.1639,  0.2779],
        [-0.1145,  0.4811],
        [-0.0395,  0.0737],
        [ 0.2236, -0.0715]], grad_fn=<MmBackward0>)


In [28]:
model_summary(gaussian_residual_model, input_size=(1, 2))

Layer (type:depth-idx)                             Output Shape              Param #
GaussianModel                                      [1, 2]                    --
├─Sequential: 1-1                                  [1, 2]                    --
│    └─GaussianFourierFeatureTransform: 2-1        [1, 4]                    --
│    └─GaussianLayer: 2-2                          [1, 512]                  --
│    │    └─Linear: 3-1                            [1, 512]                  2,560
│    │    └─gaussian_activation: 3-2               [1, 512]                  --
│    └─GaussianResidualLayer: 2-3                  [1, 512]                  --
│    │    └─Linear: 3-3                            [1, 512]                  262,656
│    │    └─gaussian_activation: 3-4               [1, 512]                  --
│    │    └─Linear: 3-5                            [1, 512]                  262,656
│    │    └─gaussian_activation: 3-6               [1, 512]                  --
│    └─GaussianResidua

In [121]:
gaussian_finer_model = GaussianFinerModel(
    in_features=2,
    out_features=2,
    scale=20.0,
    omega_f=1,
    first_k=30,
    hidden_k=30,
    residual_net=False
)

In [122]:
output = gaussian_finer_model(x_dummy)

In [123]:
print("Input:")
print(x_dummy)
print("Output:")
print(output)

Input:
tensor([[ 0.7798,  0.4665],
        [-0.5352,  0.9371],
        [ 0.6600, -0.1392],
        [ 0.7714, -0.2785],
        [-0.4264,  0.6957]])
Output:
tensor([[-0.1175, -0.1617],
        [-0.1571,  0.2604],
        [ 0.2615,  0.0056],
        [ 0.1362,  0.0373],
        [ 0.0481,  0.0955]], grad_fn=<MmBackward0>)


In [32]:
model_summary(gaussian_finer_model, input_size=(1, 2))

Layer (type:depth-idx)                             Output Shape              Param #
GaussianFinerModel                                 [1, 2]                    --
├─Sequential: 1-1                                  [1, 2]                    --
│    └─GaussianFourierFeatureTransform: 2-1        [1, 4]                    --
│    └─GaussianFinerLayer: 2-2                     [1, 128]                  --
│    │    └─Linear: 3-1                            [1, 128]                  640
│    │    └─gaussian_activation: 3-2               [1, 128]                  --
│    └─GaussianFinerLayer: 2-3                     [1, 128]                  --
│    │    └─Linear: 3-3                            [1, 128]                  16,512
│    │    └─gaussian_activation: 3-4               [1, 128]                  --
│    └─GaussianFinerLayer: 2-4                     [1, 128]                  --
│    │    └─Linear: 3-5                            [1, 128]                  16,512
│    │    └─gaussian_activ

In [33]:
gaussian_finer_residual_model = GaussianFinerModel(
    in_features=2,
    out_features=2,
    scale=30.0,
    omega_f=2.5,
    first_k=1,
    hidden_k=1,
    residual_net=True
)

In [34]:
output = gaussian_finer_residual_model(x_dummy)

In [35]:
print("Input:")
print(x_dummy)
print("Output:")
print(output)

Input:
tensor([[ 0.7798,  0.4665],
        [-0.5352,  0.9371],
        [ 0.6600, -0.1392],
        [ 0.7714, -0.2785],
        [-0.4264,  0.6957]])
Output:
tensor([[ 0.0112, -0.0602],
        [ 0.0028,  0.0391],
        [-0.0271, -0.0469],
        [-0.1279, -0.2578],
        [ 0.0530, -0.0737]], grad_fn=<MmBackward0>)


In [36]:
model_summary(gaussian_finer_residual_model, input_size=(1, 2))

Layer (type:depth-idx)                             Output Shape              Param #
GaussianFinerModel                                 [1, 2]                    --
├─Sequential: 1-1                                  [1, 2]                    --
│    └─GaussianFourierFeatureTransform: 2-1        [1, 4]                    --
│    └─GaussianFinerLayer: 2-2                     [1, 128]                  --
│    │    └─Linear: 3-1                            [1, 128]                  640
│    │    └─gaussian_activation: 3-2               [1, 128]                  --
│    └─GaussianFinerResidualLayer: 2-3             [1, 128]                  --
│    │    └─Linear: 3-3                            [1, 128]                  16,512
│    │    └─gaussian_activation: 3-4               [1, 128]                  --
│    │    └─Linear: 3-5                            [1, 128]                  16,512
│    │    └─gaussian_activation: 3-6               [1, 128]                  --
│    └─GaussianFinerResidu

## MLP MODEL

In [37]:
from models.implicit_neural_representations.inr_models.mlp_model import MLPModel

In [38]:
mlp_model = MLPModel(
    in_features=2,
    out_features=2,
    mapping_type="gauss",
    mapping_dim=4,
    mapping_scale=10,
    hidden_features=128,
    hidden_layers = 5, 
    bias=True,
    weight_init=0.1,
    bias_init=None,
    residual_net=False)

In [39]:
output = mlp_model(x_dummy)

In [40]:
print("Input:")
print(x_dummy)
print("Output:")
print(output)

Input:
tensor([[ 0.7798,  0.4665],
        [-0.5352,  0.9371],
        [ 0.6600, -0.1392],
        [ 0.7714, -0.2785],
        [-0.4264,  0.6957]])
Output:
tensor([[ 0.0125, -0.0323],
        [ 0.0107, -0.0327],
        [ 0.0121, -0.0331],
        [ 0.0107, -0.0320],
        [ 0.0116, -0.0317]], grad_fn=<AddmmBackward0>)


In [41]:
model_summary(mlp_model, input_size=(1, 2))

Layer (type:depth-idx)                             Output Shape              Param #
MLPModel                                           [1, 2]                    --
├─Sequential: 1-1                                  [1, 2]                    --
│    └─GaussianFourierFeatureTransform: 2-1        [1, 4]                    --
│    └─MLPLayer: 2-2                               [1, 128]                  --
│    │    └─Linear: 3-1                            [1, 128]                  640
│    │    └─ReLU: 3-2                              [1, 128]                  --
│    └─MLPLayer: 2-3                               [1, 128]                  --
│    │    └─Linear: 3-3                            [1, 128]                  16,512
│    │    └─ReLU: 3-4                              [1, 128]                  --
│    └─MLPLayer: 2-4                               [1, 128]                  --
│    │    └─Linear: 3-5                            [1, 128]                  16,512
│    │    └─ReLU: 3-6     

In [42]:
mlp_residual_model = MLPModel(
    in_features=2,
    out_features=2,
    mapping_type="gauss",
    mapping_dim=4,
    mapping_scale=10,
    hidden_features=128,
    hidden_layers = 5, 
    bias=True,
    weight_init=0.1,
    bias_init=None,
    residual_net=True)

In [43]:
output = mlp_residual_model(x_dummy)

In [44]:
print("Input:")
print(x_dummy)
print("Output:")
print(output)

Input:
tensor([[ 0.7798,  0.4665],
        [-0.5352,  0.9371],
        [ 0.6600, -0.1392],
        [ 0.7714, -0.2785],
        [-0.4264,  0.6957]])
Output:
tensor([[ 0.0967, -0.0440],
        [ 0.0887, -0.0052],
        [ 0.0478, -0.0944],
        [ 0.0673, -0.0025],
        [ 0.0507, -0.0650]], grad_fn=<AddmmBackward0>)


In [45]:
model_summary(mlp_residual_model, input_size=(1, 2))

Layer (type:depth-idx)                             Output Shape              Param #
MLPModel                                           [1, 2]                    --
├─Sequential: 1-1                                  [1, 2]                    --
│    └─GaussianFourierFeatureTransform: 2-1        [1, 4]                    --
│    └─MLPLayer: 2-2                               [1, 128]                  --
│    │    └─Linear: 3-1                            [1, 128]                  640
│    │    └─ReLU: 3-2                              [1, 128]                  --
│    └─MLPResidualLayer: 2-3                       [1, 128]                  --
│    │    └─Linear: 3-3                            [1, 128]                  16,512
│    │    └─ReLU: 3-4                              [1, 128]                  --
│    │    └─Linear: 3-5                            [1, 128]                  16,512
│    │    └─ReLU: 3-6                              [1, 128]                  --
│    └─MLPResidualLayer: 2

## Wire Models

In [46]:
from models.implicit_neural_representations.inr_models.wire_model import WireModel, WireFinerModel

In [47]:
wire_model = WireModel(
    in_features=2,
    out_features=2,
    mapping_type="no",
    mapping_dim=4,
    mapping_scale=10,
    hidden_features=128,
    hidden_layers = 5, 
    scale=10.0,
    omega=20.0,
    bias=True,
    weight_init=0.1,
    bias_init=None,
    residual_net=False
)

In [48]:
output = wire_model(x_dummy)

In [49]:
print("Input:")
print(x_dummy)
print("Output:")
print(output)

Input:
tensor([[ 0.7798,  0.4665],
        [-0.5352,  0.9371],
        [ 0.6600, -0.1392],
        [ 0.7714, -0.2785],
        [-0.4264,  0.6957]])
Output:
tensor([[-0.5086+0.0430j, -0.6965-0.0307j],
        [-0.3955+0.1242j, -0.3077+0.3404j],
        [-0.8281-1.2646j,  0.2292+0.1762j],
        [ 0.0733+0.7533j,  0.1020+0.3192j],
        [-0.8839+0.3014j,  0.8834-0.2571j]], grad_fn=<MmBackward0>)


In [50]:
model_summary(wire_model, input_size=(1, 2))

Layer (type:depth-idx)                   Output Shape              Param #
WireModel                                [1, 2]                    --
├─Sequential: 1-1                        [1, 2]                    --
│    └─WireLayer: 2-1                    [1, 128]                  --
│    │    └─Linear: 3-1                  [1, 128]                  384
│    │    └─wire_activation: 3-2         [1, 128]                  --
│    └─WireLayer: 2-2                    [1, 128]                  --
│    │    └─Linear: 3-3                  [1, 128]                  16,512
│    │    └─wire_activation: 3-4         [1, 128]                  --
│    └─WireLayer: 2-3                    [1, 128]                  --
│    │    └─Linear: 3-5                  [1, 128]                  16,512
│    │    └─wire_activation: 3-6         [1, 128]                  --
│    └─WireLayer: 2-4                    [1, 128]                  --
│    │    └─Linear: 3-7                  [1, 128]                  16,512
│ 

In [51]:
wire_residual_model = WireModel(
    in_features=2,
    out_features=2,
    mapping_type="gauss",
    mapping_dim=100,
    mapping_scale=10,
    hidden_features=128,
    hidden_layers = 5,
    scale=10.0,
    omega=20.0,
    bias=True,
    weight_init=0.1,
    bias_init=None,
    residual_net=True
)

In [52]:
output = wire_residual_model(x_dummy)

In [53]:
print("Input:")
print(x_dummy)
print("Output:")
print(output)

Input:
tensor([[ 0.7798,  0.4665],
        [-0.5352,  0.9371],
        [ 0.6600, -0.1392],
        [ 0.7714, -0.2785],
        [-0.4264,  0.6957]])
Output:
tensor([[ 0.0167-0.0351j, -0.0720+0.0874j],
        [ 0.0712+0.0649j,  0.1438+0.7258j],
        [ 0.4071+0.1639j, -0.0408-0.3936j],
        [ 0.2102+0.0212j,  0.1073+0.4516j],
        [-0.5808-0.2841j, -0.5139-0.5503j]], grad_fn=<MmBackward0>)


In [54]:
model_summary(wire_residual_model, input_size=(1, 2))

Layer (type:depth-idx)                             Output Shape              Param #
WireModel                                          [1, 2]                    --
├─Sequential: 1-1                                  [1, 2]                    --
│    └─GaussianFourierFeatureTransform: 2-1        [1, 100]                  --
│    └─WireLayer: 2-2                              [1, 128]                  --
│    │    └─Linear: 3-1                            [1, 128]                  12,928
│    │    └─wire_activation: 3-2                   [1, 128]                  --
│    └─WireResidualLayer: 2-3                      [1, 128]                  --
│    │    └─Linear: 3-3                            [1, 128]                  16,512
│    │    └─wire_activation: 3-4                   [1, 128]                  --
│    │    └─Linear: 3-5                            [1, 128]                  16,512
│    │    └─wire_activation: 3-6                   [1, 128]                  --
│    └─WireResidualLaye

In [55]:
torch.abs(x_dummy - output)

tensor([[0.7639, 0.5455],
        [0.6099, 1.0753],
        [0.3013, 0.4057],
        [0.5616, 0.5940],
        [0.3234, 1.3289]], grad_fn=<AbsBackward0>)

In [56]:
x_dummy - output

tensor([[ 0.7631+0.0351j,  0.5385-0.0874j],
        [-0.6064-0.0649j,  0.7934-0.7258j],
        [ 0.2528-0.1639j, -0.0984+0.3936j],
        [ 0.5612-0.0212j, -0.3859-0.4516j],
        [ 0.1544+0.2841j,  1.2096+0.5503j]], grad_fn=<SubBackward0>)

In [57]:
torch.abs(output) 

tensor([[0.0389, 0.1132],
        [0.0964, 0.7399],
        [0.4389, 0.3957],
        [0.2113, 0.4642],
        [0.6466, 0.7529]], grad_fn=<AbsBackward0>)

In [72]:
wire_finer_model = WireFinerModel(
    in_features=2,
    out_features=2,
    mapping_type="no",
    mapping_dim=4,
    mapping_scale=10,
    hidden_features=512,
    hidden_layers = 5, 
    scale=10.0,
    omega=20.0,
    omega_f=1.0,
    bias=True,
    first_k=5,
    hidden_k=5,
    residual_net=False
)

In [73]:
output = wire_finer_model(x_dummy)

In [74]:
print("Input:")
print(x_dummy)
print("Output:")
print(output)

Input:
tensor([[ 0.7798,  0.4665],
        [-0.5352,  0.9371],
        [ 0.6600, -0.1392],
        [ 0.7714, -0.2785],
        [-0.4264,  0.6957]])
Output:
tensor([[-0.0024+0.0083j,  0.0069+0.0094j],
        [ 0.0053+0.0035j,  0.0006+0.0035j],
        [-0.0043+0.0029j,  0.0013+0.0100j],
        [ 0.0074+0.0025j, -0.0022+0.0010j],
        [ 0.0053+0.0035j,  0.0006+0.0035j]], grad_fn=<MmBackward0>)


In [61]:
model_summary(wire_finer_model, input_size=(1, 2))

Layer (type:depth-idx)                   Output Shape              Param #
WireFinerModel                           [1, 2]                    --
├─Sequential: 1-1                        [1, 2]                    --
│    └─WireFinerLayer: 2-1               [1, 512]                  --
│    │    └─Linear: 3-1                  [1, 512]                  1,536
│    │    └─wire_activation: 3-2         [1, 512]                  --
│    └─WireFinerLayer: 2-2               [1, 512]                  --
│    │    └─Linear: 3-3                  [1, 512]                  262,656
│    │    └─wire_activation: 3-4         [1, 512]                  --
│    └─WireFinerLayer: 2-3               [1, 512]                  --
│    │    └─Linear: 3-5                  [1, 512]                  262,656
│    │    └─wire_activation: 3-6         [1, 512]                  --
│    └─WireFinerLayer: 2-4               [1, 512]                  --
│    │    └─Linear: 3-7                  [1, 512]                  262,6

In [114]:
wire_finer_residual_model = WireFinerModel(
    in_features=2,
    out_features=2,
    mapping_type="no",
    mapping_dim=50,
    mapping_scale=50,
    hidden_features=128,
    hidden_layers = 1, 
    scale=10,
    omega=20,
    omega_f=2.5,
    bias=True,
    first_k=2,
    hidden_k=2,
    residual_net=True
)

In [115]:
output = wire_finer_residual_model(x_dummy)

In [116]:
print("Input:")
print(x_dummy)
print("Output:")
print(output)

Input:
tensor([[ 0.7798,  0.4665],
        [-0.5352,  0.9371],
        [ 0.6600, -0.1392],
        [ 0.7714, -0.2785],
        [-0.4264,  0.6957]])
Output:
tensor([[ 0.0323-0.0994j, -0.0018-0.1072j],
        [ 0.0287+0.0197j, -0.0770+0.0391j],
        [-0.0952+0.0537j,  0.0421-0.2012j],
        [-0.1195+0.2506j,  0.1246+0.1496j],
        [ 0.0297-0.0150j,  0.0180-0.0170j]], grad_fn=<MmBackward0>)


In [65]:
model_summary(wire_finer_residual_model, input_size=(1, 2))

Layer (type:depth-idx)                   Output Shape              Param #
WireFinerModel                           [1, 2]                    --
├─Sequential: 1-1                        [1, 2]                    --
│    └─WireFinerLayer: 2-1               [1, 128]                  --
│    │    └─Linear: 3-1                  [1, 128]                  384
│    │    └─wire_activation: 3-2         [1, 128]                  --
│    └─WireFinerResidualLayer: 2-2       [1, 128]                  --
│    │    └─Linear: 3-3                  [1, 128]                  16,512
│    │    └─wire_activation: 3-4         [1, 128]                  --
│    │    └─Linear: 3-5                  [1, 128]                  16,512
│    │    └─wire_activation: 3-6         [1, 128]                  --
│    └─Linear: 2-3                       [1, 2]                    256
Total params: 33,664
Trainable params: 33,664
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.03
Input size (MB): 0.00
Forward/backw